In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather

In [2]:
import glob
import os

In [3]:
#path for 11 models
path = "/home/mlopez/EXEC/tmin_annual_45/"

In [4]:
files = list(glob.glob(os.path.join(path,'*.*')))

In [5]:
files

['/home/mlopez/EXEC/tmin_annual_45/BNU-ESM_rcp45_tn_mean_annual.nc',
 '/home/mlopez/EXEC/tmin_annual_45/INM-CM4_rcp45_tn_mean_annual.nc',
 '/home/mlopez/EXEC/tmin_annual_45/CMCC-CMS_rcp45_tn_mean_annual.nc',
 '/home/mlopez/EXEC/tmin_annual_45/IPSL-CM5A-LR_rcp45_tn_mean_annual.nc',
 '/home/mlopez/EXEC/tmin_annual_45/IPSL-CM5B-LR_rcp45_tn_mean_annual.nc',
 '/home/mlopez/EXEC/tmin_annual_45/NorESM1-M_rcp45_tn_mean_annual.nc',
 '/home/mlopez/EXEC/tmin_annual_45/CanESM2_rcp45_tn_mean_annual.nc',
 '/home/mlopez/EXEC/tmin_annual_45/GFDL-ESM2M_rcp45_tn_mean_annual.nc',
 '/home/mlopez/EXEC/tmin_annual_45/HadGEM2-CC_rcp45_tn_mean_annual.nc',
 '/home/mlopez/EXEC/tmin_annual_45/MPI-ESM-LR_rcp45_tn_mean_annual.nc',
 '/home/mlopez/EXEC/tmin_annual_45/ACCESS1-3_rcp45_tn_mean_annual.nc']

# Create DF of all models

In [6]:
dflist = []

In [8]:
for file in files:
    df = pcdf.load_as_df(file)
    dflist.append(df)

/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds['time'] = index.to_datetimeindex()
/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds['time'] = index.to_datetimeindex()
/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between

In [9]:
dflist

[                                    tn_mean
 lat       lon        time                  
 66.623306 -89.045212 1950-01-01  254.500427
                      1951-01-01  253.544449
                      1952-01-01  255.910553
                      1953-01-01  255.576248
                      1954-01-01  254.405655
 ...                                     ...
 40.041039 -74.045807 2096-01-01  281.649323
                      2097-01-01  283.469208
                      2098-01-01  282.880341
                      2099-01-01  283.279266
                      2100-01-01  282.537384
 
 [11507106 rows x 1 columns],                                     tn_mean
 lat       lon        time                  
 66.623306 -89.045212 1950-01-01  255.074188
                      1951-01-01  254.698029
                      1952-01-01  255.673157
                      1953-01-01  255.497559
                      1954-01-01  256.564209
 ...                                     ...
 40.041039 -74.045807 20

In [10]:
#Takes 3 min
%time df11 = pd.concat(dflist)

CPU times: user 3min 32s, sys: 8min 14s, total: 11min 46s
Wall time: 1min 26s


In [11]:
df11

tn_mean
lat       lon        time                  
66.623306 -89.045212 1950-01-01  254.500427
                     1951-01-01  253.544449
                     1952-01-01  255.910553
                     1953-01-01  255.576248
                     1954-01-01  254.405655
...                                     ...
40.041039 -74.045807 2096-01-01  281.487885
                     2097-01-01  281.799835
                     2098-01-01  283.148804
                     2099-01-01  282.153473
                     2100-01-01  281.282928

[126578166 rows x 1 columns]

In [12]:
dftgm = df11.reset_index()

In [13]:
dftgm

,lat,lon,time,tn_mean
0,66.623306,-89.045212,1950-01-01,254.500427
1,66.623306,-89.045212,1951-01-01,253.544449
2,66.623306,-89.045212,1952-01-01,255.910553
3,66.623306,-89.045212,1953-01-01,255.576248
4,66.623306,-89.045212,1954-01-01,254.405655
...,...,...,...,...
126578161,40.041039,-74.045807,2096-01-01,281.487885
126578162,40.041039,-74.045807,2097-01-01,281.799835
126578163,40.041039,-74.045807,2098-01-01,283.148804
126578164,40.041039,-74.045807,2099-01-01,282.153473


# Get the mean for 11 models

In [14]:
#Takes 48s
%time dftgm2 = dftgm.groupby(["lat", "lon", "time"]).mean()

CPU times: user 1min 7s, sys: 4min 44s, total: 5min 51s
Wall time: 1min 26s


In [15]:
dftgm2

tn_mean
lat       lon        time                  
40.041039 -89.045212 1950-01-01  278.676361
                     1951-01-01  278.482910
                     1952-01-01  278.417999
                     1953-01-01  278.149658
                     1954-01-01  278.387756
...                                     ...
66.623306 -61.296322 2096-01-01  263.821442
                     2097-01-01  263.411469
                     2098-01-01  264.344482
                     2099-01-01  263.893616
                     2100-01-01  263.777435

[11507106 rows x 1 columns]

# Transform temp to Celsius

In [16]:
dfC = dftgm2.copy()
dfC["tn_mean"] = dfC["tn_mean"] -273.15
    

# Define 3 periods of time: 

In [17]:
year_groups = {y:0 for y in range(1980,2011)}
year_groups.update({y:1 for y in range(2041,2071)})
year_groups.update({y:2 for y in range(2071,2101)})
#Get the mean of every period
dfC2 = dfC.reset_index()
dfp = dfC2.groupby([dfC2.time.dt.year.map(year_groups), "lat","lon"]).mean()

In [18]:
dfp.reset_index()

,time,lat,lon,tn_mean
0,0.0,40.041039,-89.045212,5.699457
1,0.0,40.041039,-88.961884,5.672854
2,0.0,40.041039,-88.878548,5.668031
3,0.0,40.041039,-88.795219,5.707927
4,0.0,40.041039,-88.711891,5.721406
...,...,...,...,...
228613,2.0,66.623306,-61.629639,-9.844964
228614,2.0,66.623306,-61.546310,-10.091969
228615,2.0,66.623306,-61.462978,-9.930800
228616,2.0,66.623306,-61.379650,-9.874316


# Historic Period

485,881/3 = 161,960. ---- instead 333,469 rows × 4 columns

In [19]:
df_h = dfp.query("time==0.0")
df_h.reset_index()

,time,lat,lon,tn_mean
0,0.0,40.041039,-89.045212,5.699457
1,0.0,40.041039,-88.961884,5.672854
2,0.0,40.041039,-88.878548,5.668031
3,0.0,40.041039,-88.795219,5.707927
4,0.0,40.041039,-88.711891,5.721406
...,...,...,...,...
76201,0.0,66.623306,-61.629639,-13.102902
76202,0.0,66.623306,-61.546310,-13.329987
76203,0.0,66.623306,-61.462978,-13.160083
76204,0.0,66.623306,-61.379650,-13.090538


In [20]:
df2050 = dfp.query("time==1.0")
df2050.reset_index()

,time,lat,lon,tn_mean
0,1.0,40.041039,-89.045212,7.686012
1,1.0,40.041039,-88.961884,7.663777
2,1.0,40.041039,-88.878548,7.663198
3,1.0,40.041039,-88.795219,7.704466
4,1.0,40.041039,-88.711891,7.724572
...,...,...,...,...
76201,1.0,66.623306,-61.629639,-10.511112
76202,1.0,66.623306,-61.546310,-10.754093
76203,1.0,66.623306,-61.462978,-10.590705
76204,1.0,66.623306,-61.379650,-10.531570


In [21]:
df2080 = dfp.query("time==2.0")
df2080.reset_index()

,time,lat,lon,tn_mean
0,2.0,40.041039,-89.045212,8.229132
1,2.0,40.041039,-88.961884,8.207193
2,2.0,40.041039,-88.878548,8.206344
3,2.0,40.041039,-88.795219,8.247128
4,2.0,40.041039,-88.711891,8.267383
...,...,...,...,...
76201,2.0,66.623306,-61.629639,-9.844964
76202,2.0,66.623306,-61.546310,-10.091969
76203,2.0,66.623306,-61.462978,-9.930800
76204,2.0,66.623306,-61.379650,-9.874316


## Import dataframe with polygons

In [22]:
dfpolyshape = from_geofeather('Grid-TerritoiresGuides.feather')

/home/mlopez/EXEC/anaconda3/envs/python37-2/lib/python3.8/site-packages/geofeather/core.py:85: UserWarning: Grid-TerritoiresGuides.feather.crs coordinate reference system file is missing. No crs will be set for this GeoDataFrame.
  warnings.warn(


In [23]:
dfpolyshape

,lat,lon,TER_GUIDE,geometry
0,52.957191,-67.712730,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
1,52.957191,-67.629402,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."
2,52.957191,-67.546066,6opqr,"POLYGON ((-67.56730 52.91553, -67.57974 52.915..."
3,52.873859,-67.796059,6opqr,"POLYGON ((-67.75439 52.89329, -67.75439 52.832..."
4,52.873859,-67.712730,6opqr,"POLYGON ((-67.75439 52.83219, -67.75439 52.893..."
...,...,...,...,...
33193,45.041668,-72.208336,2c,"POLYGON ((-72.25000 45.00450, -72.25000 45.083..."
33194,45.041668,-72.125000,2c,"POLYGON ((-72.16666 45.00508, -72.16666 45.083..."
33195,45.041668,-72.041672,2c,"POLYGON ((-72.08334 45.00564, -72.08334 45.083..."
33196,45.041668,-71.958336,2c,"POLYGON ((-72.00000 45.00667, -72.00000 45.083..."


## Merge data with mask 

In [24]:
dftg11TG = pd.merge(df_h, dfpolyshape, on=["lat","lon"])

In [25]:
dftg11TG

,lat,lon,tn_mean,TER_GUIDE,geometry
0,44.957512,-74.629120,2.390883,1a,"POLYGON ((-74.62590 44.99918, -74.58746 44.999..."
1,44.957512,-74.545792,2.335345,1a,"POLYGON ((-74.58746 44.99876, -74.58746 44.999..."
2,44.957512,-74.462463,2.283893,1a,"POLYGON ((-74.50413 44.99784, -74.50413 44.999..."
3,44.957512,-74.379128,2.243644,1a,"POLYGON ((-74.42079 44.99530, -74.42079 44.999..."
4,44.957512,-74.295799,2.184861,1a,"POLYGON ((-74.33746 44.99215, -74.33746 44.999..."
...,...,...,...,...,...
16563,52.873859,-67.629402,-7.930330,6opqr,"POLYGON ((-67.67107 52.83219, -67.67107 52.915..."
16564,52.873859,-67.546066,-7.886977,6opqr,"POLYGON ((-67.58773 52.83219, -67.58773 52.904..."
16565,52.957191,-67.712730,-8.023029,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
16566,52.957191,-67.629402,-7.934578,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."


In [26]:
dftg11TG.TER_GUIDE.value_counts()

6cdefg    2359
6opqr     1609
6ab       1420
5bcd      1231
4bc       1057
6hi       1028
6j         950
6mn        923
3ab        652
5a         552
6kl        507
5ef        401
4f         389
2b         388
3d         367
3c         356
4de        356
5g         347
5hi        342
1a         338
4gh        304
5jk        244
2c         166
2a         143
4a         139
Name: TER_GUIDE, dtype: int64

In [ ]:
TG4a = dftg11TG[dftg11TG.TER_GUIDE == "4a"]
TG4a

In [ ]:
TG4a.to_csv("/home/mlopez/EXEC/Tg_csv/TG4a11.csv")

# ---- Create GeoJson file ----

In [27]:
geometry = dftg11TG["geometry"]

In [28]:
crs = {'init': "epsg:4326"}
gdf = GeoDataFrame(dftg11TG, crs=crs, geometry=geometry)

In [ ]:
gdf.to_file("TG11tg.json", driver="GeoJSON")

In [34]:
for i in range(1, 5):
    print (i)
    print (gdf.TER_GUIDE[0])


1
1a
2
1a
3
1a
4
1a


In [29]:
gdf.TER_GUIDE[1]

'1a'